IMPORTING DEPENDENCIES


In [101]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

USING INPUT FILE

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [102]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [103]:
cap = cv2.VideoCapture("KneeBendVideo.mp4")

# Get the codec and frame rate of the input video
fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
# Get the frame size of the input video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object for the output video with the same codec, frame rate and size as the input video
out = cv2.VideoWriter('output_finaL.mp4', fourcc, fps, (width, height))
count = 0
angle_sum = 0
timer = 0
rep_count = 0
avg_angle = 0
holding_timer = 8
sum_rep_angle = 0
sum_rep_fc = 0
rep_angle = 0
rep_fc = 0
fc = 0
SOB = 0

stage = "down"

## Setup mediapipe instance
with mp_pose.Pose(model_complexity=2,min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC)

        if not ret:
            break
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip,knee,ankle)
            
            # Visualize angle
            cv2.putText(image, str(int(angle)), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA
                                )
            
            if angle < 140 and stage == "down":
                timer = current_time
                avg_angle = angle
                rep_angle = 0
                rep_fc = 0
                stage = "up"
            

            if stage == 'up':
                rep_angle = rep_angle + angle
                rep_fc = rep_fc + 1

            angle_sum = angle_sum + angle
            count = count + 1

            if count >= int(fps/2):
                avg_angle = int(angle_sum/count)
                count = 0
                angle_sum = 0
                
            if (current_time - timer)/1000 >= holding_timer and stage == "up":

                if avg_angle > 140:
                    sum_rep_angle = sum_rep_angle + rep_angle
                    sum_rep_fc = sum_rep_fc + rep_fc
                    rep_count+=1
                    stage = "down"
            else:
                

                if avg_angle > 140 and stage == "up":
                    cv2.rectangle(image, (240,0),(width-260,73),(0,0,255),-1)
                    cv2.putText(image, 'Keep Your Knee Bent', (250,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),2,cv2.LINE_AA)

                    stage = "down"

        except:
            pass
        
        tints = current_time/1000
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,200,200), -1)
        cv2.rectangle(image, (width,0), (width-225,73), (245,200,200), -1)
        cv2.rectangle(image, (width,height), (width-500,height-50), (0,200,200), -1)

        cv2.putText(image, 'Avg Angle'+':'+str(int(avg_angle)), (width-470,height-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        cv2.putText(image, 'Angle Sum:' + str(int(angle_sum)), (width-340,height-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, 'Count:'+str(count), (width-200,height-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        cv2.putText(image, 'Time:'+str(int(current_time/ 60000))+':'+str(int((current_time % 60000)/1000)), (width-100,height-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        #cv2.putText(image,str(int(tints/60))+'minutes and'+str(int(tints%60)))
        cv2.putText(image, 'Knee Rep Holding Time', (width-200,15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        #STAT OF BENDNESS(SOB)
        if sum_rep_fc > 0:
            SOB = ((sum_rep_angle/sum_rep_fc)-30)/(140-30)
        else:
            SOB = 0
    
        cv2.putText(image, 'SOB:'+  str(round(SOB,2)), (50,height-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)

        if stage == "down":
            cv2.putText(image, '0', (width-125,55), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (19,16,139), 2, cv2.LINE_AA)
            cv2.putText(image, stage, 
                    (72,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        else:
            cv2.putText(image, str(int((current_time - timer)/1000)), (width-125,63), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (19,16,139), 2, cv2.LINE_AA)
            cv2.putText(image, stage, 
                    (95,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(rep_count), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (100,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                    #(72,60), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        out.write(image)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

25.0
